<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

## Introduction

In this project, I will explore, segment, and cluster the neighborhoods in the city of Toronto. 

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Extract and Explore Dataset</a>

2. <a href="#item2">Explore Neighborhoods in New York City</a>

3. <a href="#item3">Analyze Each Neighborhood</a>

4. <a href="#item4">Cluster Neighborhoods</a>

5. <a href="#item5">Examine Clusters</a>    
</font>
</div>

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [49]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

import pgeocode # library to get the coordinates of zip codes

import requests # library to handle requests
from lxml import html # library to collect html data

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

print('Libraries imported.')

Libraries imported.


<a id='item1'></a>

## 1. Data Collection and Explore Dataset

The dataset for this project is in a Wikipedia page so the first step is to develop a little crawler and extract the data

Next, let's collect the data.

In [2]:
# Parse HTML

url_wiki = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

session = requests.session()
session.verify = False

resp = session.get(url_wiki)
root = html.fromstring(resp.content)

Let's take a quick look at the data.

In [3]:
# Function to extract the data from parsed HTML
def extract_data(root):
    row_combox = root.xpath('.//table[contains(@class, "wikitable")]/tbody/tr')
    cols = [line.text.strip().replace(" ", "_").lower() for line in list(row_combox[0])]

    toronto_data = []
    for row in row_combox[1:]:
        row_dict = {}
        for col in zip(cols, list(row)):
            row_dict[col[0]] = col[1].text.strip() if not col[1].text.strip() == 'Not assigned' else None
        
        toronto_data.append(row_dict)
    
    return toronto_data

#### Tranform the data into a *pandas* dataframe

In [4]:
toronto_data = extract_data(root)
toronto_data = pd.DataFrame(toronto_data)
toronto_data.head()

,postal_code,borough,neighbourhood
0,M1A,None,None
1,M2A,None,None
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Fix neighborhood spelling

In [5]:
toronto_data.rename(columns={'neighbourhood': 'neighborhood'}, inplace=True)
toronto_data.columns

Index(['postal_code', 'borough', 'neighborhood'], dtype='object')

#### Data Cleaning

Let's remove all rows with empty boroughs

In [6]:
# Check for NaN
nan_borough = toronto_data['borough'].isnull().sum()
print("There are 77 rows with empty boroughs.")

There are 77 rows with empty boroughs.


In [7]:
# Remove all rows with null borough 
toronto_data = toronto_data.loc[~toronto_data['borough'].isnull()].reset_index(drop=True)
toronto_data.isnull().sum()

postal_code     0
borough         0
neighborhood    0
dtype: int64

In [8]:
# Check for dataframe data types
toronto_data.head()

,postal_code,borough,neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [9]:
num_rows = toronto_data.shape[0]
print(f'There are {num_rows} rows in the Toronto dataset.')

There are 103 rows in the Toronto dataset.


Then let's loop through the postal codes and use the pgeocode library to get their respectively geographical coordenates.

In [10]:
toronto_data['latitude'] = ''
toronto_data['longitude'] = ''

for i, postal_code in enumerate(toronto_data['postal_code']):
    nomi = pgeocode.Nominatim('CA')
    results = nomi.query_postal_code(postal_code)

    toronto_data.loc[i, ['latitude', 'longitude']] = results['latitude'], results['longitude']

toronto_data.head()

,postal_code,borough,neighborhood,latitude,longitude
0,M3A,North York,Parkwoods,43.7545,-79.33
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889


Let's check for missing data in the coordenates
toronto_data.isnull().sum() 

In [11]:
toronto_data.isnull().sum()

postal_code     0
borough         0
neighborhood    0
latitude        1
longitude       1
dtype: int64

Now we are going to find the postal code with missing data and insert it manually from the spreadsheet provided

In [12]:
toronto_data.loc[toronto_data['latitude'].isnull()]

,postal_code,borough,neighborhood,latitude,longitude
76,M7R,Mississauga,Canada Post Gateway Processing Centre,NaN,NaN


In [13]:
toronto_data.loc[76, ['latitude', 'longitude']] = 43.63696, -79.6158
toronto_data.iloc[76]

postal_code                                       M7R
borough                                   Mississauga
neighborhood    Canada Post Gateway Processing Centre
latitude                                       43.637
longitude                                    -79.6158
Name: 76, dtype: object

#### Change the coordinates columns data type

In [14]:
toronto_data['latitude'] = toronto_data['latitude'].astype('float64')
toronto_data['longitude'] = toronto_data['longitude'].astype('float64')
toronto_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   postal_code   103 non-null    object 
 1   borough       103 non-null    object 
 2   neighborhood  103 non-null    object 
 3   latitude      103 non-null    float64
 4   longitude     103 non-null    float64
dtypes: float64(2), object(3)
memory usage: 4.1+ KB


Let's get number of boroughs and neighborhoods in the dataset.

In [15]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_data['borough'].unique()),
        toronto_data.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


#### Use geopy library to get the latitude and longitude values of Toronto.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.

In [16]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [17]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['latitude'], toronto_data['longitude'], toronto_data['borough'], toronto_data['neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in that contain "Toronto" in its name. So let's slice the original dataframe and create a new dataframe of the Toronto data.

In [18]:
toronto_only = toronto_data.loc[toronto_data['borough'].str.contains(pat='toronto', case=False)].reset_index(drop=True)
toronto_only.head()

,postal_code,borough,neighborhood,latitude,longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783
3,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756
4,M4E,East Toronto,The Beaches,43.6784,-79.2941


As we did with all of New York City, let's visualizat Toronto the neighborhoods in it.

In [19]:
# create map of Manhattan using latitude and longitude values from the previous map
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_only['latitude'], toronto_only['longitude'], toronto_only['neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [20]:
CLIENT_ID = 'DIYWYUU01DTOW5MNJOI32ZMRIGNNO0VAKTDABYYWSKTFK0W1' # your Foursquare ID
CLIENT_SECRET = 'JNND0D0O1Y0GNWDSZAEUFR4IUILFI14CKIJLMGVEURCNKR2V' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DIYWYUU01DTOW5MNJOI32ZMRIGNNO0VAKTDABYYWSKTFK0W1
CLIENT_SECRET:JNND0D0O1Y0GNWDSZAEUFR4IUILFI14CKIJLMGVEURCNKR2V


#### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [21]:
toronto_only.loc[0, 'neighborhood']

'Regent Park, Harbourfront'

Get the neighborhood's latitude and longitude values.

In [22]:
neighborhood_latitude = toronto_only.loc[0, 'latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_only.loc[0, 'longitude'] # neighborhood longitude value

neighborhood_name = toronto_only.loc[0, 'neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6555, -79.3626.


#### Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [23]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=DIYWYUU01DTOW5MNJOI32ZMRIGNNO0VAKTDABYYWSKTFK0W1&client_secret=JNND0D0O1Y0GNWDSZAEUFR4IUILFI14CKIJLMGVEURCNKR2V&v=20180605&ll=43.6555,-79.3626&radius=500&limit=100'

Send the GET request and examine the resutls

In [24]:
results = requests.get(url).json()
results

beledLatLngs': [{'label': 'display',
          'lat': 43.65473505045365,
          'lng': -79.35987433132891}],
        'distance': 235,
        'postalCode': 'M5A 1L9',
        'cc': 'CA',
        'city': 'Toronto',
        'state': 'ON',
        'country': 'Canada',
        'formattedAddress': ['497 King Street East (btwn Sackville St and Sumach St)',
         'Toronto ON M5A 1L9',
         'Canada']},
       'categories': [{'id': '4bf58dd8d48988d1ed941735',
         'name': 'Spa',
         'pluralName': 'Spas',
         'shortName': 'Spa',
         'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/spa_',
          'suffix': '.png'},
         'primary': True}],
       'photos': {'count': 0, 'groups': []}},
      'referralId': 'e-0-50760559e4b0e8c7babe2497-5'},
     {'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ada57aff964a520972121e3'

From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [25]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [50]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()
nearby_venues.shape

,name,categories,lat,lng
0,Tandem Coffee,Coffee Shop,43.653559,-79.361809
1,Roselle Desserts,Bakery,43.653447,-79.362017
2,Figs Breakfast & Lunch,Breakfast Spot,43.655675,-79.364503
3,The Yoga Lounge,Yoga Studio,43.655515,-79.364955
4,Sumach Espresso,Coffee Shop,43.658135,-79.359515


(22, 4)

And how many venues were returned by Foursquare?

In [27]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

22 venues were returned by Foursquare.


<a id='item2'></a>

## 2. Explore Neighborhoods in Toronto

#### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['neighborhood', 
                  'neighborhood_latitude', 
                  'neighborhood_longitude', 
                  'venue', 
                  'venue_latitude', 
                  'venue_longitude', 
                  'venue_category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [32]:
toronto_venues = getNearbyVenues(names=toronto_only['neighborhood'],
                                 latitudes=toronto_only['latitude'],
                                 longitudes=toronto_only['longitude']
                                 )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport


#### Let's check the size of the resulting dataframe

In [64]:
print(toronto_venues.shape)
toronto_venues.head()
toronto_venues.shape

(1545, 7)


,neighborhood,neighborhood_latitude,neighborhood_longitude,venue,venue_latitude,venue_longitude,venue_category
0,"Regent Park, Harbourfront",43.6555,-79.3626,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park, Harbourfront",43.6555,-79.3626,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park, Harbourfront",43.6555,-79.3626,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Regent Park, Harbourfront",43.6555,-79.3626,The Yoga Lounge,43.655515,-79.364955,Yoga Studio
4,"Regent Park, Harbourfront",43.6555,-79.3626,Sumach Espresso,43.658135,-79.359515,Coffee Shop


(1545, 7)

Let's check how many venues were returned for each neighborhood

In [51]:
toronto_venues.groupby('neighborhood').count()
toronto_venues.groupby('neighborhood').count().shape

,neighborhood_latitude,neighborhood_longitude,venue,venue_latitude,venue_longitude,venue_category
neighborhood,,,,,,
Berczy Park,91,91,91,91,91,91
"Brockton, Parkdale Village, Exhibition Place",38,38,38,38,38,38
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",14,14,14,14,14,14
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",57,57,57,57,57,57
Central Bay Street,69,69,69,69,69,69
Christie,11,11,11,11,11,11
Church and Wellesley,76,76,76,76,76,76
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,22,22,22,22,22,22


(38, 6)

#### Let's find out how many unique categories can be curated from all the returned venues

In [35]:
print('There are {} uniques categories.'.format(len(toronto_venues['venue_category'].unique())))

There are 215 uniques categories.


<a id='item3'></a>

## 3. Analyze Each Neighborhood

In [53]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['venue_category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['neighborhood'] = toronto_venues['neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()
toronto_onehot.shape

,neighborhood,Accessories Store,Afghan Restaurant,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Cafeteria,College Gym,College Rec Center,College Theater,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

(1545, 216)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [54]:
toronto_grouped = toronto_onehot.groupby('neighborhood').mean().reset_index()
toronto_grouped.head()
toronto_grouped.shape

,neighborhood,Accessories Store,Afghan Restaurant,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Cafeteria,College Gym,College Rec Center,College Theater,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.0,0.010989,0.021978,0.000000,0.000000,0.000000,0.010989,0.0,0.010989,0.032967,0.000000,0.000000,0.0,0.0,0.010989,0.000000,0.032967,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.021978,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.054945,0.0,0.0,0.000000,0.021978,0.0,0.000000,0.000000,0.021978,0.109890,0.0,0.0,0.000000,0.0,0.0,0.0,0.010989,0.000000,0.010989,0.0,0.0,0.0,0.010989,0.0,0.0,0.0,0.000000,0.021978,0.010989,0.000000,0.010989,0.000000,0.0,0.0,0.0,0.000000,0.0,0.010989,0.000000,0.0,0.0,0.0,0.021978,0.000000,0.000000,0.010989,0.0,0.0,0.010989,0.010989,0.010989,0.010989,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.010989,0.0,0.0,0.0,0.000000,0.0,0.010989,0.010989,0.010989,0.021978,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.054945,0.010989,0.0,0.0,0.0,0.0,0.0,0.021978,0.032967,0.010989,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.010989,0.010989,0.000000,0.000000,0.0,0.000000,0.000000,0

(38, 216)

#### Let's print each neighborhood along with the top 5 most common venues

In [38]:
num_top_venues = 5

for hood in toronto_grouped['neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
         venue  freq
0  Coffee Shop  0.11
1         Café  0.05
2        Hotel  0.05
3   Restaurant  0.03
4     Beer Bar  0.03


----Brockton, Parkdale Village, Exhibition Place----
                    venue  freq
0                    Café  0.08
1             Coffee Shop  0.08
2               Gift Shop  0.05
3  Thrift / Vintage Store  0.05
4          Breakfast Spot  0.05


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
            venue  freq
0      Restaurant  0.21
1     Coffee Shop  0.14
2     Yoga Studio  0.07
3  Breakfast Spot  0.07
4       Bookstore  0.07


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0         Coffee Shop  0.07
1  Italian Restaurant  0.07
2                Café  0.05
3                 Bar  0.05
4          Restaurant  0.04


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.2

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [39]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [70]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['neighborhood'] = toronto_grouped['neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()
neighborhoods_venues_sorted.shape

,neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Hotel,Café,Seafood Restaurant,Restaurant,Japanese Restaurant,Beer Bar,Bakery,Gym,Farmers Market
1,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Café,Thrift / Vintage Store,Breakfast Spot,Gift Shop,French Restaurant,Italian Restaurant,Brewery,Hawaiian Restaurant,Sandwich Place
2,"Business reply mail Processing Centre, South C...",Restaurant,Coffee Shop,Yoga Studio,Bank,Breakfast Spot,Bookstore,Sushi Restaurant,Japanese Restaurant,Italian Restaurant,Martial Arts Dojo
3,"CN Tower, King and Spadina, Railway Lands, Har...",Italian Restaurant,Coffee Shop,Bar,Café,French Restaurant,Park,Speakeasy,Gym / Fitness Center,Bank,Bakery
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Bubble Tea Shop,Middle Eastern Restaurant,Clothing Store,Breakfast Spot,Sushi Restaurant,Café,Dessert Shop


(38, 11)

## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [71]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 3, 0, 0, 0, 0])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

Before the merge let's check if both tables have all the same neighborhoods

In [72]:
toronto_only.loc[~toronto_only['neighborhood'].isin(toronto_venues['neighborhood'])]

,postal_code,borough,neighborhood,latitude,longitude
19,M5N,Central Toronto,Roselawn,43.7113,-79.4195


In [73]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'cluster_labels', kmeans.labels_)

toronto_merged = toronto_only.copy()

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('neighborhood'), on='neighborhood', how='inner')

toronto_merged.head() # check the last columns!
toronto_merged.shape

,postal_code,borough,neighborhood,latitude,longitude,cluster_labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626,0,Coffee Shop,Breakfast Spot,Yoga Studio,Thai Restaurant,Health Food Store,Italian Restaurant,Food Truck,Event Space,Electronics Store,Distribution Center
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889,0,Coffee Shop,Dance Studio,College Theater,Martial Arts Dojo,Burrito Place,Ethiopian Restaurant,Mexican Restaurant,Bubble Tea Shop,Ramen Restaurant,College Cafeteria
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Hotel,Japanese Restaurant,Italian Restaurant,Bookstore,Ramen Restaurant,Bubble Tea Shop
3,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756,0,Café,Coffee Shop,Seafood Restaurant,Restaurant,Clothing Store,Cocktail Bar,American Restaurant,Cosmetics Shop,Italian Restaurant,Farmers Market
4,M4E,East Toronto,The Beaches,43.6784,-79.2941,0,Pub,Health Food Store,Trail,Gastropub,Neighborhood,Bakery,Cheese Shop,Donut Shop,Dumpling Restaurant,Dog Run


(38, 16)

In [75]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['latitude'], toronto_merged['longitude'], toronto_merged['neighborhood'], toronto_merged['cluster_labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [77]:
toronto_merged.loc[toronto_merged['cluster_labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,borough,cluster_labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Breakfast Spot,Yoga Studio,Thai Restaurant,Health Food Store,Italian Restaurant,Food Truck,Event Space,Electronics Store,Distribution Center
1,Downtown Toronto,0,Coffee Shop,Dance Studio,College Theater,Martial Arts Dojo,Burrito Place,Ethiopian Restaurant,Mexican Restaurant,Bubble Tea Shop,Ramen Restaurant,College Cafeteria
2,Downtown Toronto,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Hotel,Japanese Restaurant,Italian Restaurant,Bookstore,Ramen Restaurant,Bubble Tea Shop
3,Downtown Toronto,0,Café,Coffee Shop,Seafood Restaurant,Restaurant,Clothing Store,Cocktail Bar,American Restaurant,Cosmetics Shop,Italian Restaurant,Farmers Market
4,East Toronto,0,Pub,Health Food Store,Trail,Gastropub,Neighborhood,Bakery,Cheese Shop,Donut Shop,Dumpling Restaurant,Dog Run
5,Downtown Toronto,0,Coffee Shop,Hotel,Café,Seafood Restaurant,Restaurant,Japanese Restaurant,Beer Bar,Bakery,Gym,Farmers Market
6,Downtown Toronto,0,Coffee Shop,Italian Restaurant,Sandwich Place,Bubble Tea Shop,Middle Eastern Restaurant,Clothing Store,Breakfast Spot,Sushi Restaurant,Café,Dessert Shop
8,Downtown Toronto,0,Café,Coffee Shop,Hotel,Gym,Restaurant,Steakhouse,American Restaurant,Salad Place,Asian Restaurant,Japanese Restaurant
9,West Toronto,0,Bakery,Grocery Store,Park,Smoke Shop,Furniture / Home Store,Middle Eastern Restaurant,Pizza Place,Bar,Bank,Café
11,West Toronto,0,Bar,Coffee Shop,Asian Restaurant,Restaurant,Vietnamese Restaurant,Cocktail Bar,Vegetarian / Vegan Restaurant,New American Restaurant,Cupcake Shop,Music Store


#### Cluster 2

In [79]:
toronto_merged.loc[toronto_merged['cluster_labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,borough,cluster_labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,West Toronto,1,Park,Yoga Studio,Dog Run,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


#### Cluster 3

In [80]:
toronto_merged.loc[toronto_merged['cluster_labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,borough,cluster_labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Central Toronto,2,Basketball Court,Park,Home Service,Trail,Yoga Studio,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


#### Cluster 4

In [81]:
toronto_merged.loc[toronto_merged['cluster_labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,borough,cluster_labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Downtown Toronto,3,Café,Grocery Store,Candy Store,Park,Coffee Shop,Playground,Baby Store,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
10,Downtown Toronto,3,Harbor / Marina,Park,Café,Music Venue,Cupcake Shop,Donut Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
23,Central Toronto,3,Garden,Gym Pool,Playground,Park,Yoga Studio,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
29,Central Toronto,3,Park,Thai Restaurant,Gym,Grocery Store,Tennis Court,Distribution Center,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
33,Downtown Toronto,3,Playground,Grocery Store,Park,Candy Store,Yoga Studio,Dog Run,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


#### Cluster 5

In [82]:
toronto_merged.loc[toronto_merged['cluster_labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,borough,cluster_labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,4,Photography Studio,Park,Distribution Center,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
